# Challenge 3

In this challenge we will work on the `Orders` data set. In your work you will apply the thinking process and workflow we showed you in Challenge 2.

You are serving as a Business Intelligence Analyst at the headquarter of an international fashion goods chain store. Your boss today asked you to do two things for her:

**First, identify two groups of customers from the data set.** The first group is **VIP Customers** whose **aggregated expenses** at your global chain stores are **above the 95th percentile** (aka. 0.95 quantile). The second group is **Preferred Customers** whose **aggregated expenses** are **between the 75th and 95th percentile**.

**Second, identify which country has the most of your VIP customers, and which country has the most of your VIP+Preferred Customers combined.**

## Q1: How to identify VIP & Preferred Customers?

We start by importing all the required libraries:

In [1]:
# import required libraries
import numpy as np
import pandas as pd

Next, extract and import `Orders` dataset into a dataframe variable called `orders`. Print the head of `orders` to overview the data:

In [2]:
# your code here
orders=pd.read_csv('/Users/barbaramuniz/Documents/GitHub/lab-dataframe-calculations/your-code/Orders.csv')
orders.head()

,Unnamed: 0,InvoiceNo,StockCode,year,month,day,hour,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,amount_spent
0,0,536365,85123A,2010,12,3,8,white hanging heart t-light holder,6,2010-12-01 08:26:00,2.55,17850,United Kingdom,15.30
1,1,536365,71053,2010,12,3,8,white metal lantern,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
2,2,536365,84406B,2010,12,3,8,cream cupid hearts coat hanger,8,2010-12-01 08:26:00,2.75,17850,United Kingdom,22.00
3,3,536365,84029G,2010,12,3,8,knitted union flag hot water bottle,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
4,4,536365,84029E,2010,12,3,8,red woolly hottie white heart.,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34


---

"Identify VIP and Preferred Customers" is the non-technical goal of your boss. You need to translate that goal into technical languages that data analysts use:

## How to label customers whose aggregated `amount_spent` is in a given quantile range?


We break down the main problem into several sub problems:

#### Sub Problem 1: How to aggregate the  `amount_spent` for unique customers?

#### Sub Problem 2: How to select customers whose aggregated `amount_spent` is in a given quantile range?

#### Sub Problem 3: How to label selected customers as "VIP" or "Preferred"?

*Note: If you want to break down the main problem in a different way, please feel free to revise the sub problems above.*

Now in the workspace below, tackle each of the sub problems using the iterative problem solving workflow. Insert cells as necessary to write your codes and explain your steps.

In [3]:
orders.drop(columns='Unnamed: 0', inplace=True)
orders.head()

,InvoiceNo,StockCode,year,month,day,hour,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,amount_spent
0,536365,85123A,2010,12,3,8,white hanging heart t-light holder,6,2010-12-01 08:26:00,2.55,17850,United Kingdom,15.30
1,536365,71053,2010,12,3,8,white metal lantern,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
2,536365,84406B,2010,12,3,8,cream cupid hearts coat hanger,8,2010-12-01 08:26:00,2.75,17850,United Kingdom,22.00
3,536365,84029G,2010,12,3,8,knitted union flag hot water bottle,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
4,536365,84029E,2010,12,3,8,red woolly hottie white heart.,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34


In [4]:
#Sub problem 1
amount_spent= orders.groupby(['CustomerID', 'Country'])['amount_spent'].sum()
amount_spent=pd.DataFrame(amount_spent)
amount_spent.reset_index()

,CustomerID,Country,amount_spent
0,12346,United Kingdom,77183.60
1,12347,Iceland,4310.00
2,12348,Finland,1797.24
3,12349,Italy,1757.55
4,12350,Norway,334.40
...,...,...,...
4342,18280,United Kingdom,180.60
4343,18281,United Kingdom,80.82
4344,18282,United Kingdom,178.05
4345,18283,United Kingdom,2094.88


In [5]:
#Sub problem 2 and Sub problem 3
amount_spent.quantile([.75, .95], axis = 0)


,amount_spent
0.75,1661.195
0.95,5774.278


In [6]:
#condition1=amount_spent['amount_spent']>5774.278 #VIP
#condition1=1661.195<amount_spent['amount_spent']<5774.278 #Preferred

def client(value):
    if value>5774.278:
        return 'VIP'
    elif 1661.195<value and value<5774.278:
        return 'Preferred'
    else:
        return 'Regular'

amount_spent['Client']=amount_spent['amount_spent'].apply(client)
amount_spent.reset_index()


,CustomerID,Country,amount_spent,Client
0,12346,United Kingdom,77183.60,VIP
1,12347,Iceland,4310.00,Preferred
2,12348,Finland,1797.24,Preferred
3,12349,Italy,1757.55,Preferred
4,12350,Norway,334.40,Regular
...,...,...,...,...
4342,18280,United Kingdom,180.60,Regular
4343,18281,United Kingdom,80.82,Regular
4344,18282,United Kingdom,178.05,Regular
4345,18283,United Kingdom,2094.88,Preferred


In [7]:
#orders_agg = orders_agg.reset_index(drop=True)
#orders_agg
#orders_agg['Type of Client'] = orders_agg["amount_spent"].map(lambda x: "VIP" if x > stats.iloc[0]["95%"] else "Preferred" if x > stats.iloc[0]["75%"] else "Not VIP nor Preferred")
#orders_agg

Now we'll leave it to you to solve Q2 & Q3, which you can leverage from your solution for Q1:

## Q2: How to identify which country has the most VIP Customers?

In [8]:
# your code here
VIP_Customers=amount_spent.groupby(['Country', 'Client']).agg(quantity=('Client', 'count'))
VIP_Customers=VIP_Customers.reset_index()
VIP_Customers

VIP_Customers_Final=VIP_Customers[VIP_Customers['Client']=='VIP'].sort_values(by='quantity', ascending=False).head(1)

VIP_Customers_Final

,Country,Client,quantity
73,United Kingdom,VIP,178


## Q3: How to identify which country has the most VIP+Preferred Customers combined?

In [9]:
# your code here
orders_pivot = VIP_Customers.pivot_table(values='quantity', index='Country', columns='Client', aggfunc='first').fillna(0)
orders_pivot['VIP+Preferred'] = orders_pivot['Preferred'] + orders_pivot['VIP']
orders_pivot.sort_values(by='VIP+Preferred',ascending=False).head()

#https://pandas.pydata.org/docs/reference/api/pandas.pivot_table.html

Client,Preferred,Regular,VIP,VIP+Preferred
Country,,,,
United Kingdom,756.0,2987.0,178.0,934.0
Germany,28.0,55.0,11.0,39.0
France,20.0,58.0,9.0,29.0
Belgium,11.0,14.0,0.0,11.0
Switzerland,6.0,12.0,3.0,9.0


In [10]:
#another solution
#vip_preferred = amout_client[(amout_client["customer_type"] == "VIP") | (amout_client["customer_type"] == "Preferred")].groupby("Country").agg({"CustomerID": "count"})
#vip_preferred.sort_values("CustomerID", ascending= False).head()

In [11]:
#trying another solution
VIP_Customers_OHE = pd.get_dummies(VIP_Customers['Client'])
VIP_Customers_OHE

,Preferred,Regular,VIP
0,1,0,0
1,0,1,0
2,0,0,1
3,1,0,0
4,0,1,0
...,...,...,...
70,0,1,0
71,1,0,0
72,0,1,0
73,0,0,1
